In [1]:
from bs4 import BeautifulSoup as soup 
from urllib.request import urlopen as uReq
from urllib.request import Request
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

In [11]:
myurl =  'https://cedh-decklist-database.com'
cards = []

In [12]:
client = uReq(myurl) # opening connection 
page_html = client.read() # content to variable
client.close() # closes connection 
page_soup = soup(page_html, 'html.parser') # html parsing 
containers = page_soup.findAll("ul",{"class": "ddb-decklists"})
htmls = []
for c in containers:
    htmls.append(c.a["href"])

In [13]:
for decklist in htmls:
    chromedriver_path= r"C:\Users\Brian\OneDrive\Desktop\chromedriver.exe"
    driver = webdriver.Chrome(chromedriver_path)
    driver.get(decklist)
    time.sleep(3) #if you want to wait 3 seconds for the page to load
    page_source = driver.page_source
    driver.close()
    page_soup = soup(page_source, 'lxml')
    containers = page_soup.findAll("tr",{"class": "table-deck-row"})
    for c in containers:
        cards.append( c.a.get_text())

In [63]:
card_data = pd.DataFrame({'Card Name': cards})
card_data.to_csv('cards.csv')

In [67]:
import pandas as pd
import numpy as np
card_data = pd.read_csv('cards.csv', usecols = ['0'])
card_data  = card_data.rename({'0': 'Card Name'}, axis = 1)
card_data = pd.DataFrame(card_data.value_counts())
card_data.rename( columns={0 :'Occurances'}, inplace=True )
card_data = card_data[card_data['Occurances'] > 10]
card_data.to_csv("CEDH_Staple_Ranking.csv")